<a href="https://colab.research.google.com/github/ria3999/mahine_learning-codes/blob/master/HR_ANALYTICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import os

In [3]:
DeprecationWarning('ignore')
warnings.filterwarnings('ignore',message="ignore it")

In [4]:
df=pd.read_csv('hr_train.csv')

In [5]:
def others(df):
    df=df.drop(['employee_id','recruitment_channel','gender'],axis=1)
    df['previous_year_rating'].fillna(3.0,inplace=True)
    df['education'].fillna("Bachelor's",inplace=True)
    df['work_ratio']=df['length_of_service']/df['age']
    df['start_year']=df['age']-df['length_of_service']
    df['total_performance']=df['previous_year_rating']+df['KPIs_met >80%']+df['awards_won?']+df['avg_training_score']
    df['training_score']=df['avg_training_score']*df['no_of_trainings']
    df['total_performance'].fillna(df['total_performance'].mean(),inplace=True)
    return(df)

In [6]:
def bini(df):
    bins1=[20,35,45,60]
    label1=['young','adult','old']
    df['age']=pd.cut(df['age'],bins1,labels=label1,include_lowest=True)
    return(df)

In [7]:
def label_encode(df):
    from sklearn.preprocessing import LabelEncoder
    label=LabelEncoder()
    df['department']=label.fit_transform(df['department'])
    df['region']=label.fit_transform(df['region'])
    df['education']=label.fit_transform(df['education'])
    df['age']=label.fit_transform(df['age'])
    return(df)

In [8]:
df=others(df)
df=bini(df)
df=label_encode(df)


In [9]:
#from sklearn.model_selection import train_test_split
#train,test=train_test_split(df,test_size=0.2,random_state=19)

In [10]:
def x_and_y(df):
    Y=df['is_promoted']
    X=df.drop(['is_promoted'],axis=1)
    return X,Y

In [11]:
X_train,Y_train=x_and_y(df)

In [12]:
df1=pd.read_csv('hr_test.csv')
ids=df1['employee_id']
df1=others(df1)
df1=bini(df1)
df1=label_encode(df1)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
clf = GradientBoostingClassifier(n_estimators=510)
clf.fit(X_train, Y_train)
pred1= clf.predict(df1)


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
dt_model=DecisionTreeClassifier(max_depth=12)
dt_model.fit(X_train,Y_train)
pred2=dt_model.predict(df1)

In [15]:
from sklearn.ensemble import RandomForestClassifier
rand_model=RandomForestClassifier(n_estimators=79,max_depth=42,random_state=35)
rand_model.fit(X_train,Y_train)
pred3=rand_model.predict(df1)

In [16]:
from statistics import mode
final=np.array([])
for i in range(0,len(df1)):
    final=np.append(final,mode([pred1[i],pred2[i],pred3[i]]))

In [17]:
Submission5=pd.DataFrame({'employee_id':ids,'is_promoted':pred1})
Submission5.to_csv("submission5.csv",sep=",",index=["employee_id","is_promoted"])